In [1]:
from util import load_data, fdr, plot_report
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from collections import ChainMap
import matplotlib.pyplot as plt

In [2]:
x_train, x_test, x_oot, y_train, y_test, y_oot = load_data('var_30.csv')
x_train.head()

,ssn_name_count_14,ssn_name_dob_count_14,ssn_lastname_count_14,ssn_firstname_count_14,ssn_dob_count_0_by_30,name_dob_count_0_by_30,fulladdress_homephone_count_14,fulladdress_count_1_by_14,fulladdress_homephone_day_since,ssn_dob_day_since,...,fulladdress_count_0,name_dob_count_3,fulladdress_homephone_count_0_by_14,name_day_since,name_count_14,ssn_dob_count_0_by_14,ssn_lastname_count_0_by_14,ssn_firstname_count_0_by_14,ssn_name_day_since,address_count_1_by_7
263942,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.646013,-0.652729,...,-0.041351,-0.046415,0.120387,-0.430216,-0.159376,0.118004,0.123458,0.123528,-0.636839,0.103441
62505,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-1.358033,-1.364306,...,-0.041351,-0.046415,0.120387,-1.139211,-0.159376,0.118004,0.123458,0.123528,-1.350029,0.103441
627190,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,0.607141,0.599646,...,-0.041351,-0.046415,0.120387,0.817616,-0.159376,0.118004,0.123458,0.123528,0.618375,0.103441
369646,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,-0.285257,-0.292197,...,-0.041351,-0.046415,0.120387,-0.070992,-0.159376,0.118004,0.123458,0.123528,-0.275489,0.103441
768561,-0.070962,-0.068281,-0.071051,-0.071088,0.162987,0.163303,-0.070089,0.123861,1.100808,1.093006,...,-0.041351,-0.046415,0.120387,1.309185,-0.159376,0.118004,0.123458,0.123528,1.112853,0.103441


In [4]:
dt = DecisionTreeClassifier()
params = {'min_samples_leaf': [128, 256, 512, 1024],
          'max_depth' : [10, 20, 40, 80],
          'ccp_alpha': [0, 1e-8, 1e-7, 5e-7, 1e-6]}
grid = GridSearchCV(dt, params, cv=10, refit='FDR', 
                    verbose=1, n_jobs=6,
                    scoring={'FDR': fdr})
grid.fit(x_train, y_train)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   35.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  3.9min
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  9.4min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed: 16.2min
[Parallel(n_jobs=6)]: Done 800 out of 800 | elapsed: 16.4min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=6,
             param_grid={'ccp_alpha': [0, 1e-08, 1e-07, 5e-07, 1e-06],
         

In [5]:
print(f"The best parameters are {grid.best_params_} "
      f"with a score of {grid.best_score_:0.5f}")

The best parameters are {'ccp_alpha': 1e-07, 'max_depth': 40, 'min_samples_leaf': 128} with a score of 0.52432


In [15]:
pd.set_option('display.max_rows', 999)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_FDR').filter(
        items=['param_max_depth', 'param_min_samples_leaf',
               'param_ccp_alpha', 'mean_test_FDR', 'std_test_FDR',
               'rank_test_FDR', 'mean_test_ROC', 'std_test_ROC'])

,param_max_depth,param_min_samples_leaf,param_ccp_alpha,mean_test_FDR,std_test_FDR,rank_test_FDR
40,40,128,1e-07,0.524318,0.012785,1
44,80,128,1e-07,0.524231,0.012896,2
36,20,128,1e-07,0.523273,0.014121,3
4,20,128,0,0.523185,0.014340,4
20,20,128,1e-08,0.523098,0.014269,5
42,40,512,1e-07,0.521965,0.014364,6
46,80,512,1e-07,0.521965,0.014364,6
37,20,256,1e-07,0.521965,0.015224,8
30,80,512,1e-08,0.521530,0.013915,9
26,40,512,1e-08,0.521530,0.013915,9


In [16]:
dt = DecisionTreeClassifier(ccp_alpha=1e-07, max_depth=40, min_samples_leaf=128)
params = {'class_weight': ['balanced', None],
          'criterion': ['gini', 'entropy']}
grid_2 = GridSearchCV(dt, params, cv=10, refit='FDR', 
                      verbose=1, n_jobs=4,
                      scoring={'FDR': fdr, 'ROC': 'roc_auc'})
grid_2.fit(x_train, y_train)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  40 out of  40 | elapsed:  1.4min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=1e-07,
                                              class_weight=None,
                                              criterion='gini', max_depth=40,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=128,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=4,
             param_grid={'class_

In [19]:
print(f"The best parameters are {grid_2.best_params_} "
      f"with a score of {grid_2.best_score_:0.5f}")

The best parameters are {'class_weight': None, 'criterion': 'gini'} with a score of 0.52432


In [20]:
pd.DataFrame(grid_2.cv_results_).filter(
    items=['param_class_weight', 'param_criterion',
           'mean_test_FDR', 'std_test_FDR', 'rank_test_FDR',
           'mean_test_ROC', 'std_test_ROC'])

,param_class_weight,param_criterion,mean_test_FDR,std_test_FDR,rank_test_FDR,mean_test_ROC,std_test_ROC
0,balanced,gini,0.521006,0.014502,3,0.768478,0.008264
1,balanced,entropy,0.520570,0.015722,4,0.767968,0.008865
2,None,gini,0.524318,0.012785,1,0.769425,0.008891
3,None,entropy,0.522052,0.014966,2,0.769151,0.009340


In [22]:
dt = DecisionTreeClassifier(ccp_alpha=1e-07, 
                            max_depth=40, 
                            min_samples_leaf=128, 
                            class_weight=None,
                            criterion='gini').fit(x_train, y_train)
print(fdr(dt, x_train, y_train))
print(fdr(dt, x_test, y_test))

0.5366108786610879
0.5302978432043821


In [25]:
x_train_40, x_test_40, y_train_40, y_test_40 = load_data('var_40.csv')
dt = DecisionTreeClassifier(ccp_alpha=1e-07, 
                            max_depth=40, 
                            min_samples_leaf=128, 
                            class_weight=None,
                            criterion='gini').fit(x_train_40, y_train_40)
print(fdr(dt, x_train_40, y_train_40))
print(fdr(dt, x_test_40, y_test_40))

0.5711297071129707
0.5515234508729887
